In [2]:
%pprint
%load_ext supriya.ext.ipython

import warnings

warnings.filterwarnings(action="default", module="supriya")

from supriya.scsynth import kill

kill()

Pretty printing has been turned ON
The supriya.ext.ipython extension is already loaded. To reload it, use:
  %reload_ext supriya.ext.ipython


In [ ]:
# soundcheck

March 18th, 2025

# Supriya: a Python API for SuperCollider

Joséphine Wolf Oberholtzer (she / her)

## Introduction

- who am i
- what is supriya
- what isn't it
- what is this presentation
- what isn't this presentation
- what's in a name
- presentation overview

## Servers

### Import supriya

In [ ]:
import supriya

In [ ]:
dir(supriya)

### Import `Server`

In [ ]:
from supriya import Server

In [ ]:
server = Server()

In [ ]:
dir(server)

### Server options

In [ ]:
server

In [ ]:
# TODO: Make Options a dataclass
server.options

### Boot the server

In [ ]:
server.boot()

### Inspect the server

In [ ]:
server.status

In [ ]:
print(tree := server.query_tree())

In [ ]:
tree

### Reset the server

In [ ]:
server.reset()

In [ ]:
print(tree := server.query_tree())

### Quit the server

In [ ]:
server.quit()

### Boot with options

In [ ]:
server.boot(maximum_logins=2)

### Multiple users

In [ ]:
other_server = Server()
other_server

In [ ]:
server

In [ ]:
other_server.connect()

In [ ]:
other_server.disconnect()

In [ ]:
server

### Lifecycle events

In [ ]:
def on_event(event):
    print(event)

for event_type in supriya.ServerLifecycleEvent:
    server.on(event_type, on_event)
    other_server.on(event_type, on_event)

In [ ]:
server.reboot()

In [ ]:
# this will panic
other_server.boot()

In [ ]:
server.quit()

## Entities

### Groups

In [ ]:
server = Server().boot()

In [ ]:
# add a group
group = server.add_group()
print(group)
print(dir(group))
print(server.query_tree())

In [ ]:
# add a group to the group
child_group = group.add_group()
print(server.query_tree())

In [ ]:
# move the child group into the default group
child_group.move(target_node=server.default_group, add_action="ADD_TO_TAIL")
print(server.query_tree())

In [ ]:
# free the original parent group
group.free()
print(server.query_tree())

### Synths

In [ ]:
# add a synth (this will fail, silently)
synth = child_group.add_synth(synthdef=supriya.default)

In [ ]:
# we have a proxy to a synth
synth

In [ ]:
# but nothing on the server, because the request failed
print(server.query_tree())

In [ ]:
# using on_completion keyword argument
server.add_synthdefs(
    supriya.default,
    on_completion=lambda server: child_group.add_synth(synthdef=supriya.default),
) 
server.sync()
print(server.query_tree())

In [ ]:
# using completion context manager
with server.at():
    with server.add_synthdefs(supriya.default):
        synth = child_group.add_synth(synthdef=supriya.default, frequency=666)
server.sync()
print(server.query_tree())

In [ ]:
synth.set(frequency=555)

In [ ]:
synth.free()

In [ ]:
child_group.set(gate=0)

### Buses

- calculation rates
- allocate and free
- buses vs bus groups
- shared memory

### Buffers

- allocate and free
- read sound files
- generating
- plotting

## Messages

In [ ]:
server.reset()

In [ ]:
server.osc_protocol

In [ ]:
with server.osc_protocol.capture() as transcript:
    with server.at():
        with server.add_synthdefs(supriya.default):
            group = server.add_group()
            synth = group.add_synth(synthdef=supriya.default, frequency=666)

In [ ]:
for x in transcript: print(x)

### OSC messages & bundles

### Sending messages

### OSC protocols

### Capturing IO

### Moments & completions

### Requests & responses

## Synthdefs

### Building SynthDefs

### `synthdef` decorator

### Graphing SynthDefs

### SynthDef internals

### Envelopes

### Done actions

## Clocks & Patterns

## Non-Realtime

- server
- session
- mutations vs queries
- rendering (async)

## Concurrency

- server
- async server
- threaded concurrency
- event loop / coroutine concurrency

## Mixers

## Cleanliness

- docs
- typing
- testing
- ci/cd